In [1]:
# %pip install numpy
# %pip install pandas
# %pip install matplotlib
# %pip install seaborn
# %pip install statsmodels
# %pip install -U scikit-learn scipy matplotlib

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels import stats
import scipy
import matplotlib.pyplot as plt

def mean_absolute_error(y, yp):
    mae = sum(abs(y - yp)) / len(y)
    return mae

df = pd.read_csv('MLclass_PISAData.csv', encoding='gbk', index_col=['CNTSTUID'])
data = df.copy()
#print (data.describe())
data.pop('CNTRYID')
data.pop('CNT')
data.pop('CNTSCHID')
data.pop('SN')
#print(data.columns)
for col in data.columns:
    if data[col].dtypes== 'object':
        data[col]=pd.to_numeric(data[col],errors='coerce')

data[np.isnan(data)] = 0

#print(data.dtypes)

In [3]:
target = data.pop('PVSCIE')

# https://reurl.cc/mMoMWM
# Statistic
data = sm.add_constant(data)
Statistic = sm.OLS(target,data).fit()
print(Statistic.summary())
print()

# Residual
res = target - Statistic.fittedvalues
#print(res)

                            OLS Regression Results                            
Dep. Variable:                 PVSCIE   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 2.751e+04
Date:                Sun, 12 May 2024   Prob (F-statistic):               0.00
Time:                        23:07:05   Log-Likelihood:                -28345.
No. Observations:                7708   AIC:                         5.675e+04
Df Residuals:                    7679   BIC:                         5.695e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7123      0.855      0.833      0.4

In [4]:
max_performance_test = 0
order = 0

#建立線性回歸時，必須要檢驗殘差(Residaul)是否符合以下三個假設：
#	• 常態性 ( Normality )
#	• 獨立性 ( Independence )
#	• 變異數同質性 ( Homogeneity of Variance )

# Normalty 
# Not Normal distribution, if p-value < 0.05  
#print (scipy.stats.shapiro(res))
#if (scipy.stats.shapiro(res).pvalue < 0.05) : print ('The Residual is not normally distributed')
#else : print ('The Residual is normally distributed')
#print()

res_pvalue = scipy.stats.mstats.normaltest(res)
print('P value =', res_pvalue.pvalue)
if (res_pvalue.pvalue < 0.05) : print ('The Residual is not normally distributed.')
else : print ('The Residual is normally distributed.')
print()

# Sample Inependence
# 每個殘差之間互為獨立：可使用 Durban-Watson test 檢定每個殘差之間是否獨立。
# p-value > 0.05：接受虛無假設，代表每個殘差之間互為獨立。
tstatistic = stats.stattools.durbin_watson(res) # 計算出 T 統計量
print('Durbin-Watson Test :', tstatistic )
tpvalue = scipy.stats.t.sf(tstatistic, len(res)-1) # 將統計量轉換成 p-value
print('t-statistic =', tstatistic)
print('P-Value =', tpvalue)
if (tpvalue < 0.05) : print ('The Residual is not independent.')
else : print ('The Residual is independent.')
print()

# Variance Homogeneity
# 殘差變異數一致性：使用 Breusch-Pagan Test 檢測殘差變異數是否一致
#  p-value < 0.05：拒絕虛無假設，代表殘差變異數不一致。
print('P-Value of Variance Hemogeneity =', sms.het_breuschpagan(res, Statistic.model.exog)[3])
if (sms.het_breuschpagan(res, Statistic.model.exog)[3] < 0.05) : print ('The Variance of Residual is not Homogeneity.')
else : print ('The Variance of Residual is Homogeneity.')
print()
    

mean_performance_train = []
mean_performance_test = []
mean_MAE = []

run_times = 100

for i in range (run_times):
    # start training and testing the multiple regression model
    from sklearn.model_selection import train_test_split
    data_train, data_test, target_train, target_test = train_test_split(data.values, target.values, test_size=0.2)
    
    from sklearn.linear_model import LinearRegression
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.pipeline import make_pipeline
    
    #reg_model = LinearRegression()
    
    #reg_model.fit(data_train, target_train)
    poly_model = make_pipeline(PolynomialFeatures(2), LinearRegression())
    poly_model.fit(data_train, target_train)
    
    #predictions = reg_model.predict(data_test)
    predictions = poly_model.predict(data_test)

    mean_MAE.append(mean_absolute_error(target_test,predictions))
    mean_performance_train.append(poly_model.score(data_train, target_train))
    mean_performance_test.append(poly_model.score(data_test, target_test))

print( 'Mean MAE = ', np.mean(mean_MAE).round(3))
print( 'Std 0f MAE = ', np.std(mean_MAE).round(3))
print()
print( 'Mean Score of Train Set = ', np.mean(mean_performance_train).round(3))
print( 'Std Score of Train Set = ', np.std(mean_performance_train).round(3))
print()
print( 'Mean Score of Test Set = ', np.mean(mean_performance_test).round(3))
print( 'Std Score of Test Set = ', np.std(mean_performance_test).round(3))
print()


P value = 0.04628030069180523
The Residual is not normally distributed.

Durbin-Watson Test : 2.0061659621546384
t-statistic = 2.0061659621546384
P-Value = 0.022436710855715743
The Residual is not independent.

P-Value of Variance Hemogeneity = 0.0001506730997862439
The Variance of Residual is not Homogeneity.

Mean MAE =  8.327
Std 0f MAE =  0.606

Mean Score of Train Set =  0.99
Std Score of Train Set =  0.002

Mean Score of Test Set =  0.986
Std Score of Test Set =  0.006

